# Setup Ports

# Port Utilities and Boundary Operations

This notebook demonstrates utilities for working with maritime ports and geographic boundaries.

Learn how to:
- Query port information from the included port database (~7000 global ports)
- Format port data for nautical displays
- Create geographic boundaries around ports
- Use boundaries for spatial filtering of ENC data

## Port Database
The toolkit includes a comprehensive port database with:
- **Location**: Latitude/longitude coordinates
- **Characteristics**: Size class, shelter type, anchorage depth
- **Services**: Fuel, repairs, provisions, water, electricity
- **Communications**: VHF, radio, phone, fax, air contact
- **Total Ports**: ~7000 global ports from the World Port Index (WPI)

Access this data via the `PortData` class.

**Data Source**: The port database is sourced from the World Port Index (WPI), 
originally compiled by the U.S. National Geospatial-Intelligence Agency (NGA). 
This database is in the public domain and contains no restrictions on use.

## Boundary Creation
Use the `Boundaries` class to create expanded geographic zones around ports for:
- Defining areas of interest (AOI) for data filtering
- Establishing approach corridors
- Creating fairway buffers
- Handling International Date Line crossing (for Pacific/Asia-Pacific routes)

Boundaries are returned as GeoDataFrame with polygon geometry, suitable for spatial queries and map visualization.

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# --- Example Port Names ---
# Change these to query different ports
example_port_1 = "Los Angeles"
example_port_2 = "San Francisco"
example_port_3 = "Miami"

# =============================================================================
# SETUP AND IMPORTS
# =============================================================================

import sys
import os
from pathlib import Path
from dotenv import load_dotenv
import plotly.io as pio

# Project setup
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.nautical_graph_toolkit.utils.port_utils import *
from src.nautical_graph_toolkit.utils.plot_utils import PlotlyChart

# Initialize port database
port = PortData()

# Initialize Plotly Chart with Mapbox token validation
mapbox_token = os.getenv('MAPBOX_TOKEN')
if not mapbox_token:
    print("⚠️  MAPBOX_TOKEN not found in .env file")
    print("   Maps will use free OpenStreetMap basemap.")
    print("   For custom Mapbox styles, add MAPBOX_TOKEN to your .env file")
else:
    print("✓ MAPBOX_TOKEN found - using custom Mapbox basemap")

pio.renderers.default = "notebook_connected"
ply = PlotlyChart()
ply_fig = ply.create_base_map(mapbox_token=mapbox_token)
ply.plotly_base_config(ply_fig)

print("=" * 70)
print("✓ Port database loaded successfully")
print("=" * 70)
print(f"Total ports available: {len(port.get_port_names())}")
print("=" * 70)

## List of Ports

Query the port database to find all available ports.
Results include both standard ports and custom user-defined ports (if configured).


In [ ]:
port.get_port_names()

## Port Details: Retrieve Information for a Single Port

Access complete information for a specific port, including:
- Index number and port name
- Exact latitude/longitude coordinates
- Port characteristics (size, shelter type, entry restrictions)
- Available services (fuel, repairs, provisions, etc.)
- Communication facilities (VHF, radio, phone, etc.)

**Note**: Port names are case-insensitive. If a port is not found, check spelling or use `port.get_port_names()` to see all available ports.

In [ ]:
port_name = example_port_2

# Look up port with error handling
try:
    idx = port.get_port_index(port_name)
    port_series = port.get_port_by_name(port_name)
    
    if port_series is None:
        raise ValueError(f"Port '{port_name}' not found")
    
    print(f"✓ Found port at index {idx}")
    print(f"  {port.format_port_string(port_series)}")
    
except ValueError as e:
    print(f"❌ {e}")
    print(f"\nAvailable ports starting with '{port_name[0].upper()}':")
    matching = [p for p in port.get_port_names() if p.upper().startswith(port_name[0].upper())][:10]
    print(f"  {', '.join(matching)}")
    raise

print("\n")
port_series

## Format Port Information as String

Convert port data to a human-readable nautical format:
- Port name and country code
- Position in degrees, minutes, and cardinal directions

Useful for logging, reporting, and display.


In [ ]:
port.format_port_string(port_series)

## Step 1: Visualize Single Port on Map

Display the selected port on an interactive map to view its location.
Use pan and zoom to explore the surrounding area.

In [ ]:
ply.add_single_port_trace(ply_fig, port_series, name=port_series['PORT_NAME'], color='blue')
ply_fig.show()

## Cleaned Series into Dataframe

In [ ]:
port.get_port_details_df(port_series)

## Step 2: Create Geographic Boundaries

Create expanded geographic boundaries around ports for spatial filtering.

### Use Cases
- **Area of Interest (AOI)**: Define region for ENC data selection
- **Buffer Zones**: Expand ports to include surrounding approach areas
- **Route Corridors**: Create boundaries between departure and arrival ports
- **Date Line Handling**: Automatically handle boundaries crossing 180° meridian

Boundaries are returned as GeoDataFrame with polygon geometry, suitable for
spatial queries and map visualization.

In [ ]:
try:
    # Retrieve port data
    port1 = port.get_port_by_name(example_port_1)
    port2 = port.get_port_by_name(example_port_2)
    port3 = port.get_port_by_name(example_port_3)
    
    # Validate all ports were found
    ports = [port1, port2, port3]
    port_names = [example_port_1, example_port_2, example_port_3]
    missing = [name for name, p in zip(port_names, ports) if p is None]
    
    if missing:
        print(f"❌ Failed to find ports: {', '.join(missing)}")
        raise ValueError(f"Missing ports: {missing}")
    
    # Create boundary
    bbox = Boundaries()
    port_bbox = bbox.create_geo_boundary(
        geometries=[port1.geometry, port2.geometry, port3.geometry]
    )
    
    print(f"✓ Created boundary around {len(ports)} ports:")
    for name, p in zip(port_names, ports):
        print(f"  - {name}: {p['PORT_NAME']}")
    
    print(f"\nBoundary properties:")
    print(f"  - Geometry type: {port_bbox.geometry.iloc[0].geom_type}")
    print(f"  - Area (degrees²): {port_bbox.geometry.iloc[0].area:.2f}")
    print(f"  - Bounds: {port_bbox.geometry.iloc[0].bounds}")
    
    port_bbox
    
except Exception as e:
    print(f"❌ Failed to create boundary: {e}")
    raise

## Step 3: Visualize Boundary on Map

Display the geographic boundary on the map to see the area of interest
that encompasses all selected ports.

In [ ]:
ply.add_single_port_trace(ply_fig, port3, name=port3['PORT_NAME'], color='red')
ply.add_boundary_trace(ply_fig, port_bbox)
ply_fig.show()